# 共同研究者を管理する

GRDMプロジェクトの共同研究者の追加や削除、権限の管理を行う。

## 共同編集者の表示
GRDMに登録されている共同研究者を表示します。<br>

・アクセストークン<br>
・プロジェクトID（暫定）<br>
・表示（ボタン）<br>

共同研究者の編集はGRDMのプロジェクトのメニューにあるメンバーから行ってください。<br>
※【注意】メンバーの権限が「読込み/書込み」「管理者」のいずれかに設定されているメンバーでないとリサーチフローは操作できません。<br>

In [ ]:
import os
import sys
import traceback
from requests.exceptions import RequestException

import panel as pn
import pandas as pd
from IPython.display import display
from IPython.core.display import Javascript, HTML

sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector
from library.utils.widgets import Button, MessageBox
from library.utils.config import message as msg_config
from library.utils.error import InputWarning, UnauthorizedError
from library.utils.storage_provider import grdm
from library.utils.checker import StringManager
from library.utils.html.button import create_button

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'


class CollaboratorManager(TaskDirector):

    def __init__(self, working_path:str) -> None:
        super().__init__(working_path, notebook_name)

        # フォームボックス
        self._form_box = pn.WidgetBox()
        self._form_box.width = 900
        # メッセージ用ボックス
        self._msg_output = MessageBox()
        self._msg_output.width = 900

        self.show_col = ShowCollaborator(self._form_box, self._msg_output)

    @TaskDirector.task_cell("1")
    def generateFormScetion(self):
        # タスク開始によるサブフローステータス管理JSONの更新
        self.doing_task(script_file_name)

        # フォーム定義
        self.show_col.define_form()
        self.show_col.set_submit_button_callback(self.show_collaborator_callback)
        # フォーム表示
        pn.extension()
        form_section = pn.WidgetBox()
        form_section.append(self._form_box)
        form_section.append(self._msg_output)
        display(form_section)


    @TaskDirector.callback_form("show_collaborator")
    def show_collaborator_callback(self, event):
        self.show_col.submit_button.set_looks_processing()
        self._msg_output.clear()
        try:
            self.show_col.get_collaborators()
        except InputWarning as e:
            self.log.warning(str(e))
            return
        except RequestException as e:
            self.log.error(str(e))
            return
        except Exception:
            message = f'## [INTERNAL ERROR] : {traceback.format_exc()}'
            self.show_col.submit_button.set_looks_error(msg_config.get('DEFAULT', 'unexpected_error'))
            self._msg_output.update_error(message)
            self.log.error(message)
            return

        # タスク実行の完了情報を該当サブフローステータス管理JSONに書き込む
        self.done_task(script_file_name)


class ShowCollaborator:

    def __init__(self, form_box, message_box:MessageBox) -> None:
        self._form_box = form_box
        self._msg_output = message_box

        self.project_id = grdm.get_project_id()
        # define widgets
        self.token_title = msg_config.get('form', 'token_title')
        self.token_form = pn.widgets.PasswordInput(
            name=self.token_title,
            width=600
        )
        self.project_form = pn.widgets.TextInput(name="Project ID", width=600)
        self.submit_button = Button(width=600)
        self.link_button = pn.pane.HTML()

    def define_form(self):
        # display
        self._form_box.append(self.token_form)
        if not self.project_id:
            self._form_box.append(self.project_form)
        self.submit_button.set_looks_init()
        self._form_box.append(self.submit_button)
        if self.project_id:
            self.link_button.object = create_button(
            url=grdm.get_collaborator_url(grdm.SCHEME, grdm.DOMAIN, self.project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            button_width='500px'
        )
            self._form_box.append(self.link_button)

    def set_submit_button_callback(self, func):
        self.submit_button.on_click(func)

    def get_collaborators(self):
        """メイン処理"""
        token = self.token_form.value_input

        try:
            # token
            token = StringManager.strip(token, remove_empty=True)
            if not token:
                raise InputWarning(msg_config.get('form', 'none_input_value').format(self.token_title))
            if StringManager.has_whitespace(token):
                raise InputWarning(msg_config.get('form', 'token_invalid'))
            # priject id
            if not self.project_id:
                project_id = self.project_form.value_input
                project_id = StringManager.strip(project_id)
                if StringManager.is_empty(project_id):
                    message = msg_config.get('form', 'none_input_value').format("Project ID")
                self.project_id = project_id
        except InputWarning as e:
            self.submit_button.set_looks_warning(str(e))
            raise

        try:
            contents = grdm.get_collaborator_list(grdm.SCHEME, grdm.API_DOMAIN, token, self.project_id)
        except UnauthorizedError:
            message = msg_config.get('form', 'token_unauthorized')
            self.submit_button.set_looks_warning(message)
            raise InputWarning(message)
        except RequestException:
            message = msg_config.get('DEFAULT', 'connection_error')
            self.submit_button.set_looks_error(message)
            raise
        except Exception:
            message = msg_config.get('DEFAULT', 'unexpected_error')
            self.submit_button.set_looks_error(message)
            raise

        self.display_table(contents)
        self.link_button.object = create_button(
            url=grdm.get_collaborator_url(grdm.SCHEME, grdm.DOMAIN, self.project_id),
            msg=msg_config.get('manage_collaborators', 'go_grdm'),
            button_width='500px'
        )
        self._form_box.append(self.link_button)


    def display_table(self, contents:dict):
        display_permission = {
                "admin": msg_config.get('manage_collaborators', 'admin'),
                "write": msg_config.get('manage_collaborators', 'write'),
                "read": msg_config.get('manage_collaborators', 'read')
            }

        name_list = []
        permission_list = []
        for full_name, permission in contents.items():
            name_list.append(full_name)
            permission_list.append(display_permission[permission])

        df = pd.DataFrame({
            msg_config.get('manage_collaborators', 'name_title'): name_list,
            msg_config.get('manage_collaborators', 'permission_title'): permission_list
        })
        self._form_box.clear()
        table = pn.pane.DataFrame(df, index=False)
        self._form_box.append('# 共同研究者')
        self._form_box.append(table)


CollaboratorManager(working_path=os.path.abspath('__file__')).generateFormScetion()

## サブフローメニューへ

サブフローメニューへアクセスするボタンを表示します

In [ ]:
import os
import sys
sys.path.append('../../../../..') # 研究準備
sys.path.append('../../../../../..') # 研究準備以外
from library.task_director import TaskDirector

script_file_name = "manage_collaborators"
notebook_name = script_file_name+'.ipynb'
TaskDirector(os.path.abspath('__file__'), notebook_name).return_subflow_menu()